# Reference Tune-up

This notebook shall evolve into a reference tune-up notebook. Currently, it is to showcase the new experiment library and the related helper files and functions.

In [9]:
# Convenience Import for all LabOne Q Functionality
from laboneq.simple import *

import experiment_library.experiments.experiment_library as experiments
from experiment_library.calibration_helpers import load_qubit_parameters, update_qubit_parameters_and_calibration

In [10]:
emulate = True

In [ ]:
from textwrap import dedent

descriptor = dedent(
    """\
instruments:
  SHFQC:
    - address: dev12250
      uid: device_shfqc
connections:

  device_shfqc:
    - iq_signal: q0/measure_line
      ports: QACHANNELS/0/OUTPUT
    - acquire_signal: q0/acquire_line
      ports: QACHANNELS/0/INPUT
      
    - iq_signal: q0/drive_line
      ports: SGCHANNELS/3/OUTPUT
"""
)
demo_setup = DeviceSetup.from_descriptor(
    descriptor, server_host="localhost", server_port=8004, setup_name="MySetup"
)

## Apply Calibration from File

In [11]:
qubit_parameters = load_qubit_parameters()

qubit_parameters["local_oscillators"]["readout_lo"]["value"] = 7.0e9

transmon_list = update_qubit_parameters_and_calibration(
    qubit_parameters, device_setup=demo_setup, database = None
)

### Create and Connect to a QCCS Session 

Establishes the connection to the instruments and readies them for experiments


In [12]:
# create and connect to a session 
session = Session(device_setup=demo_setup)
session.connect(do_emulation=emulate, reset_devices=True)

[2023.10.10 20:05:04.261] INFO    Logging initialized from [Default inline config in laboneq.controller.laboneq_logging] logdir is c:\Users\andreasm\Downloads\laboneq-library\example\laboneq_output\log
[2023.10.10 20:05:04.265] INFO    VERSION: laboneq 2.16.0
[2023.10.10 20:05:04.266] INFO    Connecting to data server at localhost:8004
[2023.10.10 20:05:04.267] INFO    Connected to Zurich Instruments LabOne Data Server version 23.06 at localhost:8004
[2023.10.10 20:05:04.269] INFO    Configuring the device setup
[2023.10.10 20:05:04.272] INFO    The device setup is configured


# Tune-up

## Amplitude Rabi Experiment

In [13]:
amplitude_sweep = LinearSweepParameter(start=0, stop=1, count=5)
num_averages = 2**10
cal_trace = False

amplitude_rabi = experiments.amplitude_rabi_single(
                            qubit=transmon_list[0], 
                            amplitude_sweep=amplitude_sweep,
                            num_averages = num_averages,
                            cal_trace = cal_trace )

In [ ]:
res = session.run(amplitude_rabi)

In [ ]:
show_pulse_sheet("Rabi", res.compiled_experiment)